In [20]:
import numpy as np
import sys
from casadi import *

# Import do_mpc package:
import do_mpc
import math 

In [21]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

In [22]:
#state variables
eta = model.set_variable(var_type='_x', var_name='eta', shape=(6,1))
v = model.set_variable(var_type='_x', var_name='v', shape=(6,1))

#Velocity vector
# eta_dot = model.set_variable(var_type='_z', var_name='eta_dot', shape=(6,1))
# v_dot = model.set_variable(var_type='_z', var_name='v_dot', shape=(6,1))

# Input
thrust = model.set_variable(var_type='_u', var_name='thrust', shape=(6,1))

In [23]:
thrust

SX([thrust_0, thrust_1, thrust_2, thrust_3, thrust_4, thrust_5])

Paarameters

In [24]:
x_g = 0
y_g = 0
z_g = 0
x_b = 0
y_b = 0
z_b = -0.01

#added mass coefficients
X_ud = 6.356673886738176
Y_vd = 7.120600295756984
Z_wd = 18.686326861534997
K_pd = 0.185765630747592
M_qd = 0.134823349429660
N_rd = 0.221510466644690
#drag 
Xu = {"Linear": 13.7, "NonLinear": 141}
Yv = {"Linear": 0, "NonLinear": 217}
Zw = {"Linear": 33, "NonLinear": 190}
Kp = {"Linear": 0, "NonLinear": 1.192}
Mq = {"Linear": 0.8, "NonLinear": 0.470}
Nr = {"Linear": 0, "NonLinear": 1.5}


In [25]:
m = 13.5 # Mass of the Robot

I_x, I_y, I_z = 0.26, 0.23, 0.37 # Mass Moments of Inertia

L_h, L_w, L_l = 0.378, 0.575, 0.457 

A_F, A_S, A_T = 0.0877, 0.1131, 0.2049

Volume = 0.0135

Ic = np.array([[0.26, 0, 0],
      [0, 0.23, 0],
      [0, 0, 0.37]])

Ap_F, Ap_S, Ap_T = 0.1727, 0.2174, 0.2628

Mass matrix

In [26]:
#Rigid-body mass
M_RB = np.array([
    [m, 0, 0, 0, 0, 0],
    [0, m, 0, 0, 0, 0],
    [0, 0, m, 0, 0, 0],
    [0, 0, 0, I_x, 0, 0 ],
    [0, 0, 0, 0, I_y, 0 ],
    [0, 0, 0, 0, 0, I_z ]
    ])
#added mass
M_A = -1 * np.array([[X_ud, 0, 0, 0, 0, 0],
              [0, Y_vd, 0, 0, 0, 0],
              [0, 0, Z_wd, 0, 0, 0],
              [0, 0, 0, K_pd, 0, 0 ],
              [0, 0, 0, 0, M_qd, 0 ],
              [0, 0, 0, 0, 0, N_rd ]])
M = M_RB + M_A

Coriolis

In [27]:
#the rigid body Coriolis
# v = [u v w p q r]
C_RB = np.array([
    [0, 0, 0, 0, m*v[2], -m*v[1] ],
    [0, 0, 0, -m*v[2], 0, m*v[0] ],
    [0, 0, 0, m*v[1], -m*v[0], 0 ],
    [0, m*v[2], -m*v[1], 0, -I_z*v[5], -I_y*v[4] ],
    [-m*v[2], 0, m*v[0], I_z*v[5], 0, I_x*v[3] ],
    [m*v[1], -m*v[0], 0, I_y*v[4], -I_x*v[3], 0 ],
     ])

# added mass Coriolis
C_A = np.array([
    [0, 0, 0, 0, -Z_wd*v[2], Y_vd*v[1]],
    [0, 0, 0, Z_wd*v[2], 0,-X_ud*v[0]], 
    [0, 0, 0, -Y_vd*v[1], X_ud*v[0], 0],
    [0, -Z_wd*v[2], Y_vd*v[1], 0, -N_rd*v[5], M_qd*v[4]],
    [Z_wd*v[2], 0, -X_ud*v[0], N_rd*v[5], 0, -K_pd*v[3]],
    [-Y_vd*v[1], X_ud*v[0], 0, -M_qd*v[4], K_pd*v[3], 0],
    ])

C = C_RB + C_A

Drag

In [28]:
D_lin = np.array([
    [Xu["Linear"], 0, 0, 0, 0, 0],
    [0, Yv["Linear"], 0, 0, 0, 0],
    [0, 0, Zw["Linear"], 0, 0, 0],
    [0, 0, 0, Kp["Linear"], 0, 0],
    [0, 0, 0, 0, Mq["Linear"], 0],
    [0, 0, 0, 0, 0, Nr["Linear"]]
])
    
D_nonlin = np.array([
    [Xu["NonLinear"] * v[0], 0, 0, 0, 0, 0 ],
    [0, Yv["NonLinear"] * v[1], 0, 0, 0, 0 ],
    [0, 0, Zw["NonLinear"] * v[2], 0, 0, 0 ],
    [0, 0, 0, Kp["NonLinear"] * v[3], 0, 0 ],
    [0, 0, 0, 0, Mq["NonLinear"] * v[4], 0 ],
    [0, 0, 0, 0, 0, Nr["NonLinear"] * v[5] ],        
])

D = D_lin + D_nonlin

G and B

In [29]:
W = m * 9.81
B = 1000 * Volume * 9.81

# eta = [x y z phi theta epsi]
phi = eta[3]
theta = eta[4]
psi = eta[5]

In [30]:
G = vertcat(
    (W - B) * sin(theta),
    -(W - B) * cos(theta) * sin(phi),
    -(W - B) * cos(theta) * cos(phi),
    -(y_g*W - y_b*B)*cos(theta)*cos(phi) + (z_g*W - z_b*B)*cos(theta)*sin(phi),
    (z_g*W - z_b*B)*sin(theta) + (x_g*W - x_b*B)*cos(theta)*cos(phi),
    -(x_g*W - x_b*B)*cos(theta)*sin(phi) - (y_g*W - y_b*B)*sin(theta)
    )

J

In [31]:
#kinematic equation

J2 = [
    [1, sin(phi)*tan(theta), cos(phi)*tan(theta)],
    [0, cos(phi), -sin(phi)],
    [0, sin(phi)/cos(theta), cos(phi)/cos(theta)]
]

J1 = [
    [cos(psi)*cos(theta), -sin(psi)*cos(phi) + cos(psi)*sin(theta)*sin(phi), sin(psi)*sin(phi) + cos(psi)*cos(phi)*sin(theta)],
    [sin(psi)*cos(theta), cos(psi)*cos(phi) + sin(phi)*sin(theta)*sin(psi), -cos(psi)*sin(phi) + sin(theta)*sin(psi)*cos(phi)],
    [-sin(theta), cos(theta)*sin(phi), cos(theta)*cos(phi)]
]

J = np.array([
    [cos(psi)*cos(theta), -sin(psi)*cos(phi) + cos(psi)*sin(theta)*sin(phi), sin(psi)*sin(phi) + cos(psi)*cos(phi)*sin(theta), 0, 0, 0],
    [sin(psi)*cos(theta), cos(psi)*cos(phi) + sin(phi)*sin(theta)*sin(psi), -cos(psi)*sin(phi) + sin(theta)*sin(psi)*cos(phi), 0, 0, 0],
    [-sin(theta), cos(theta)*sin(phi), cos(theta)*cos(phi), 0, 0, 0],
    [0, 0, 0, 1, sin(phi)*tan(theta), cos(phi)*tan(theta)],
    [0, 0, 0, 0, cos(phi), -sin(phi)],
    [0, 0, 0, 0, sin(phi)/cos(theta), cos(phi)/cos(theta)]

])

In [32]:
eta_dot = J @ v
v_dot =  inv( M ) @ (thrust - C @ v - D @ v - G )

Right Hand Side

In [33]:
model.set_rhs('eta', eta_dot)
model.set_rhs('v', v_dot)

In [34]:
model.setup()

Controller

In [35]:
mpc = do_mpc.controller.MPC(model)

Testing using custom solver

In [36]:
ipopt_options = {
    'verbose': False, \
    "ipopt.tol": 1e-4,
    "ipopt.acceptable_tol": 1e-4,
    "ipopt.max_iter": 1000,
    "ipopt.warm_start_init_point": "yes",
    "ipopt.print_level": 0, 
    "print_time": False
}

# self.solver = ca.nlpsol("solver", "ipopt", nlp_dict, ipopt_options)
# # jit (just-in-time compilation)
# print("Generating shared library........")
# cname = self.solver.generate_dependencies("mpc_v1.c")  
# system('gcc -fPIC -shared -O3 ' + cname + ' -o ' + self.so_path) # -O3

# # reload compiled mpc
so_path = "./saved/mpc_v1.so"
print(so_path)
solver = nlpsol("solver", "ipopt",so_path)

./saved/mpc_v1.so


RuntimeError: .../casadi/core/function_internal.cpp:145: Error calling IpoptInterface::init for 'solver':
Error in Function::factory for 'nlp' [External] at .../casadi/core/function.cpp:1758:
Failed to create nlp_hess_l:[x, p, lam:f, lam:g]->[triu:hess:gamma:x:x] with {"gamma": [f, g]}:
.../casadi/core/external.cpp:466: Assertion "s_io[i] == ret.name_out(i)" failed:
Inconsistent output name. Expected: [triu_hess_gamma_x_x], got hess_gamma_x_x for output 0

In [ ]:
setup_mpc = {
    'n_horizon': 10,
    'n_robust': 2,
    'open_loop': 0,
    't_step': 0.1,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 3,
    'collocation_ni': 1,
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    'nlpsol_opts': {'ipopt.solve': 'solver'}
}
mpc.set_param(**setup_mpc)


In [ ]:
# cost matrix for tracking the goal point
Q_goal = np.diag([
    100, 100, 100,  # delta_x, delta_y, delta_z
    10, 10, 10, # delta_qw, delta_qx, delta_qy
    10, 10, 10,
    10, 10, 10])
 
# cost matrix for the action
Q_weights = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
Q_u = np.diag(Q_weights) # T, wx, wy, wz
x_desired = np.array([
    0, 0, 0,
    0, 0, 0,
    0, 0, 0,
    0, 0, 0
])

x_current = vertcat(eta, v)

Delta_s = x_current - x_desired

In [ ]:
Delta_s

In [ ]:
cost_goal = Delta_s.T @ Q_goal @ Delta_s 
# cost_u = thrust.T @ Q_u @ thrust

mterm = cost_goal
lterm = cost_goal

mpc.set_objective(mterm=mterm, lterm=lterm)
mpc.set_rterm(thrust=Q_weights)

In [ ]:
cost_goal.shape

In [ ]:
max_states = np.array([inf, inf, inf, pi, pi, pi])
min_states = -1 * np.array([inf, inf, 0, pi, pi, pi])
# lower bounds of the states
mpc.bounds['lower','_x','eta'] = min_states
mpc.bounds['upper','_x','eta'] = max_states

# lower bounds of the input
mpc.bounds['lower','_u','thrust'] = -20 * np.ones([6,1])
mpc.bounds['upper','_u','thrust'] =  20 * np.ones([6,1])

In [ ]:
mpc.setup()

Estimator

In [ ]:
estimator = do_mpc.estimator.StateFeedback(model)

Simulator

In [ ]:
simulator = do_mpc.simulator.Simulator(model)

In [ ]:
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    'integration_tool': 'idas',
    'abstol': 1e-8,
    'reltol': 1e-8,
    't_step': 0.04
}

simulator.set_param(**params_simulator)

In [ ]:
simulator.setup()

In [ ]:
# Seed
np.random.seed(99)

# Initial state
e = np.ones([model.n_x,1])
x0 = np.random.uniform(-3*e,3*e) # Values between +3 and +3 for all states
mpc.x0 = x0
simulator.x0 = x0
estimator.x0 = x0

# Use initial state to set the initial guess.
mpc.set_initial_guess()

In [ ]:
u0 = mpc.make_step(x0)

In [ ]:
%%capture
for k in range(50):
    u0 = mpc.make_step(x0)
    print("u0:", u0)
    # y_next = simulator.make_step(u0)
    # x0 = estimator.make_step(y_next)

Displaying the results

In [ ]:
from matplotlib import rcParams
rcParams['axes.grid'] = True
rcParams['font.size'] = 18

In [ ]:
import matplotlib.pyplot as plt
fig, ax, graphics = do_mpc.graphics.default_plot(mpc.data, figsize=(16,9))
graphics.plot_results()
graphics.reset_axes()
plt.show()